#Import Libary

In [ ]:
import json, re, string, datetime, tweepy, requests
import numpy as np, pandas as pd, matplotlib.pyplot as plt, seaborn as sns
import nltk, gensim, tqdm
from bs4 import BeautifulSoup
from tabulate import tabulate
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.lancaster import LancasterStemmer
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from wordcloud import WordCloud
from textblob import TextBlob

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

from gensim import corpora
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# Dataset

In [ ]:
df_test = pd.read_csv('/content/question (1).csv')
df_test

,Unnamed: 0,text,category,product_name,product_id
0,0,Warnanya tdk sesuai pesanan. Tdk ada chat/pemb...,elektronik,USB HUB 3.0 7 port by DIGIGEAR HIGH SPEED 1.2 ...,170197447
1,1,Thanks gan barang bagus.smoga awet,handphone,Holder anti hujan &amp; copet untuk smartphone...,37564148
2,2,barang sesuai deskripsi.. laptop dapat di-charge,elektronik,Adaptor Charger Laptop Toshiba Satellite C800 ...,254339251
3,3,Goood product.......... ............,handphone,Silikon Case Blackberry Bold 9000 Hitam Gratis...,4837401
4,4,Bagus pengiriman cepat.,handphone,Audio Splitter Jack 3.5mm to dual female U Sha...,270447288
5,5,Terima kasih pesanannya sdh sampai,fashion,Celana jeans anak pendek,294837423
6,6,"barangnya cacat, apa tidak diperiksa dulu?",handphone,Tempelan / Perekat Gurita Hp Buat / untuk Goje...,56486193
7,7,Barang sesuai pesanan,elektronik,Baterai Acer Aspire One 722 D255 D257 D260 D27...,189413883
8,8,"Udah order ternyata stok ga tersedia, harusny...",fashion,Sepatu Bata Dairy Black - 6516353,194223236
9,9,Pengiriman barang lambat,elektronik,USB HUB 4 port USB 3.0 / USB HUB 3.0 &#40;4por...,182937734


In [ ]:
df_train = pd.read_csv('/content/review_product (1).csv')
df_train.head()

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,text,rating,category,product_name,product_id,sold,shop_id,product_url
0,0,0,1,Barang sesuai pesanan dan cepat sampai,5,pertukangan,Staples Dekorasi Staples Kayu + Refill 8mm - S...,418660637,1,1740837,https://www.tokopedia.com/shakaonline87/staple...
1,1,1,2,Barang bagus harga murah,5,pertukangan,STAPLE GUN ATS 3 WAY TACKER - STAPLES JOK TEMB...,416032545,11,1477109,https://www.tokopedia.com/juraganperkakas/stap...
2,2,2,3,Paket rapi...mantap....cepat....sampe ke tujuan,5,pertukangan,STAPLE GUN ATS 3 WAY TACKER - STAPLES JOK TEMB...,416032545,11,1477109,https://www.tokopedia.com/juraganperkakas/stap...
3,3,3,4,ya saya puas dgn barangnya,5,pertukangan,ALAT STAPLES TEMBAK &#40;AIR NAILER GUN&#41; O...,102279869,5,771395,https://www.tokopedia.com/kamarmesin/alat-stap...
4,4,4,5,Responya luar biasa b mantap,5,pertukangan,Isi Refill Staples Jok Kulit Motor / Staple Gu...,190679689,787,969999,https://www.tokopedia.com/mitrapersada/isi-ref...


In [ ]:
df_test.shape

(10, 5)

In [ ]:
df_train.shape

(40597, 11)

# Pre Processing

In [ ]:
def transform_ratings(rating):
    if rating == 5 or rating == 4:
        return "positive"
    if rating == 3:
        return "neutral"
    if rating == 2 or rating == 1:
        return "negative"

In [ ]:
df_train['rating'] = df_train['rating'].apply(transform_ratings)

In [ ]:
df_train.rename(columns={"rating":"sentiment"}, inplace=True)
drop_col = ['Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0', 'category', 'product_name', 'product_id', 'sold', 'shop_id', 'product_url']
df_train.drop(columns=drop_col, inplace=True)

In [ ]:
df_train.head()

,text,sentiment
0,Barang sesuai pesanan dan cepat sampai,positive
1,Barang bagus harga murah,positive
2,Paket rapi...mantap....cepat....sampe ke tujuan,positive
3,ya saya puas dgn barangnya,positive
4,Responya luar biasa b mantap,positive


###case folding

In [ ]:
def casefolding(text):
  # lower case
  text = text.lower()

  # menghilangkan tag, tautan, dan angka
  text = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^A-Za-z \t])|(\w+:\/\/\S+)|(\d+)", " ", text).split())

  # menghilangkan tanda baca
  text = text.translate(str.maketrans("","",string.punctuation))

  # Menghilangkan whitespaces
  text = text.strip()

  return text

In [ ]:
df_train['clean'] = df_train['text'].apply(casefolding)
df_test['clean'] = df_test['text'].apply(casefolding)

###remove emoji

In [ ]:
def remove_emojis(teks):
    pola_emoji = re.compile(
        "["
        "\U0001F600-\U0001F64F"  # Emotikon
        "\U0001F300-\U0001F5FF"  # Simbol dan Piktograf Lainnya
        "\U0001F680-\U0001F6FF"  # Simbol Transportasi & Peta
        "\U0001F700-\U0001F77F"  # Pelengkap Alfanumerik
        "\U0001F780-\U0001F7FF"  # Bentuk Geometri yang Diperpanjang
        "\U0001F800-\U0001F8FF"  # Panah Pelengkap-C
        "\U0001F900-\U0001F9FF"  # Simbol dan Piktograf Pelengkap
        "\U0001FA00-\U0001FA6F"  # Simbol Catur
        "\U0001FA70-\U0001FAFF"  # Simbol dan Piktograf Diperpanjang-A
        "\U00002702-\U000027B0"  # Dingbats
        "\U000024C2-\U0001F251"
        "]+",
        flags=re.UNICODE,
    )

    teks_tanpa_emoji = pola_emoji.sub(r"", teks)

    return teks_tanpa_emoji

In [ ]:
df_train['clean'] = df_train['clean'].apply(remove_emojis)
df_test['clean'] = df_test['clean'].apply(remove_emojis)

###remove emoticons

In [ ]:
EMOTICONS = {
    u":‑\)":"Happy face or smiley",
    u":\)":"Happy face or smiley",
    u":-\]":"Happy face or smiley",
    u":\]":"Happy face or smiley",
    u":-3":"Happy face smiley",
    u":3":"Happy face smiley",
    u":->":"Happy face smiley",
    u":>":"Happy face smiley",
    u"8-\)":"Happy face smiley",
    u":o\)":"Happy face smiley",
    u":-\}":"Happy face smiley",
    u":\}":"Happy face smiley",
    u":-\)":"Happy face smiley",
    u":c\)":"Happy face smiley",
    u":\^\)":"Happy face smiley",
    u"=\]":"Happy face smiley",
    u"=\)":"Happy face smiley",
    u":‑D":"Laughing, big grin or laugh with glasses",
    u":D":"Laughing, big grin or laugh with glasses",
    u"8‑D":"Laughing, big grin or laugh with glasses",
    u"8D":"Laughing, big grin or laugh with glasses",
    u"X‑D":"Laughing, big grin or laugh with glasses",
    u"XD":"Laughing, big grin or laugh with glasses",
    u"=D":"Laughing, big grin or laugh with glasses",
    u"=3":"Laughing, big grin or laugh with glasses",
    u"B\^D":"Laughing, big grin or laugh with glasses",
    u":-\)\)":"Very happy",
    u":‑\(":"Frown, sad, andry or pouting",
    u":-\(":"Frown, sad, andry or pouting",
    u":\(":"Frown, sad, andry or pouting",
    u":‑c":"Frown, sad, andry or pouting",
    u":c":"Frown, sad, andry or pouting",
    u":‑<":"Frown, sad, andry or pouting",
    u":<":"Frown, sad, andry or pouting",
    u":‑\[":"Frown, sad, andry or pouting",
    u":\[":"Frown, sad, andry or pouting",
    u":-\|\|":"Frown, sad, andry or pouting",
    u">:\[":"Frown, sad, andry or pouting",
    u":\{":"Frown, sad, andry or pouting",
    u":@":"Frown, sad, andry or pouting",
    u">:\(":"Frown, sad, andry or pouting",
    u":'‑\(":"Crying",
    u":'\(":"Crying",
    u":'‑\)":"Tears of happiness",
    u":'\)":"Tears of happiness",
    u"D‑':":"Horror",
    u"D:<":"Disgust",
    u"D:":"Sadness",
    u"D8":"Great dismay",
    u"D;":"Great dismay",
    u"D=":"Great dismay",
    u"DX":"Great dismay",
    u":‑O":"Surprise",
    u":O":"Surprise",
    u":‑o":"Surprise",
    u":o":"Surprise",
    u":-0":"Shock",
    u"8‑0":"Yawn",
    u">:O":"Yawn",
    u":-\*":"Kiss",
    u":\*":"Kiss",
    u":X":"Kiss",
    u";‑\)":"Wink or smirk",
    u";\)":"Wink or smirk",
    u"\*-\)":"Wink or smirk",
    u"\*\)":"Wink or smirk",
    u";‑\]":"Wink or smirk",
    u";\]":"Wink or smirk",
    u";\^\)":"Wink or smirk",
    u":‑,":"Wink or smirk",
    u";D":"Wink or smirk",
    u":‑P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"X‑P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"XP":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":‑Þ":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":Þ":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":b":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"d:":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"=p":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u">:P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":‑/":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":/":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":-[.]":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u">:[(\\\)]":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u">:/":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":[(\\\)]":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u"=/":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u"=[(\\\)]":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":L":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u"=L":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":S":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":‑\|":"Straight face",
    u":\|":"Straight face",
    u":$":"Embarrassed or blushing",
    u":‑x":"Sealed lips or wearing braces or tongue-tied",
    u":x":"Sealed lips or wearing braces or tongue-tied",
    u":‑#":"Sealed lips or wearing braces or tongue-tied",
    u":#":"Sealed lips or wearing braces or tongue-tied",
    u":‑&":"Sealed lips or wearing braces or tongue-tied",
    u":&":"Sealed lips or wearing braces or tongue-tied",
    u"O:‑\)":"Angel, saint or innocent",
    u"O:\)":"Angel, saint or innocent",
    u"0:‑3":"Angel, saint or innocent",
    u"0:3":"Angel, saint or innocent",
    u"0:‑\)":"Angel, saint or innocent",
    u"0:\)":"Angel, saint or innocent",
    u":‑b":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"0;\^\)":"Angel, saint or innocent",
    u">:‑\)":"Evil or devilish",
    u">:\)":"Evil or devilish",
    u"\}:‑\)":"Evil or devilish",
    u"\}:\)":"Evil or devilish",
    u"3:‑\)":"Evil or devilish",
    u"3:\)":"Evil or devilish",
    u">;\)":"Evil or devilish",
    u"\|;‑\)":"Cool",
    u"\|‑O":"Bored",
    u":‑J":"Tongue-in-cheek",
    u"#‑\)":"Party all night",
    u"%‑\)":"Drunk or confused",
    u"%\)":"Drunk or confused",
    u":-###..":"Being sick",
    u":###..":"Being sick",
    u"<:‑\|":"Dump",
    u"\(>_<\)":"Troubled",
    u"\(>_<\)>":"Troubled",
    u"\(';'\)":"Baby",
    u"\(\^\^>``":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    u"\(\^_\^;\)":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    u"\(-_-;\)":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    u"\(~_~;\) \(・\.・;\)":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    u"\(-_-\)zzz":"Sleeping",
    u"\(\^_-\)":"Wink",
    u"\(\(\+_\+\)\)":"Confused",
    u"\(\+o\+\)":"Confused",
    u"\(o\|o\)":"Ultraman",
    u"\^_\^":"Joyful",
    u"\(\^_\^\)/":"Joyful",
    u"\(\^O\^\)／":"Joyful",
    u"\(\^o\^\)／":"Joyful",
    u"\(__\)":"Kowtow as a sign of respect, or dogeza for apology",
    u"_\(\._\.\)_":"Kowtow as a sign of respect, or dogeza for apology",
    u"<\(_ _\)>":"Kowtow as a sign of respect, or dogeza for apology",
    u"<m\(__\)m>":"Kowtow as a sign of respect, or dogeza for apology",
    u"m\(__\)m":"Kowtow as a sign of respect, or dogeza for apology",
    u"m\(_ _\)m":"Kowtow as a sign of respect, or dogeza for apology",
    u"\('_'\)":"Sad or Crying",
    u"\(/_;\)":"Sad or Crying",
    u"\(T_T\) \(;_;\)":"Sad or Crying",
    u"\(;_;":"Sad of Crying",
    u"\(;_:\)":"Sad or Crying",
    u"\(;O;\)":"Sad or Crying",
    u"\(:_;\)":"Sad or Crying",
    u"\(ToT\)":"Sad or Crying",
    u";_;":"Sad or Crying",
    u";-;":"Sad or Crying",
    u";n;":"Sad or Crying",
    u";;":"Sad or Crying",
    u"Q\.Q":"Sad or Crying",
    u"T\.T":"Sad or Crying",
    u"QQ":"Sad or Crying",
    u"Q_Q":"Sad or Crying",
    u"\(-\.-\)":"Shame",
    u"\(-_-\)":"Shame",
    u"\(一一\)":"Shame",
    u"\(；一_一\)":"Shame",
    u"\(=_=\)":"Tired",
    u"\(=\^\·\^=\)":"cat",
    u"\(=\^\·\·\^=\)":"cat",
    u"=_\^=	":"cat",
    u"\(\.\.\)":"Looking down",
    u"\(\._\.\)":"Looking down",
    u"\^m\^":"Giggling with hand covering mouth",
    u"\(\・\・?":"Confusion",
    u"\(?_?\)":"Confusion",
    u">\^_\^<":"Normal Laugh",
    u"<\^!\^>":"Normal Laugh",
    u"\^/\^":"Normal Laugh",
    u"\（\*\^_\^\*）" :"Normal Laugh",
    u"\(\^<\^\) \(\^\.\^\)":"Normal Laugh",
    u"\(^\^\)":"Normal Laugh",
    u"\(\^\.\^\)":"Normal Laugh",
    u"\(\^_\^\.\)":"Normal Laugh",
    u"\(\^_\^\)":"Normal Laugh",
    u"\(\^\^\)":"Normal Laugh",
    u"\(\^J\^\)":"Normal Laugh",
    u"\(\*\^\.\^\*\)":"Normal Laugh",
    u"\(\^—\^\）":"Normal Laugh",
    u"\(#\^\.\^#\)":"Normal Laugh",
    u"\（\^—\^\）":"Waving",
    u"\(;_;\)/~~~":"Waving",
    u"\(\^\.\^\)/~~~":"Waving",
    u"\(-_-\)/~~~ \($\·\·\)/~~~":"Waving",
    u"\(T_T\)/~~~":"Waving",
    u"\(ToT\)/~~~":"Waving",
    u"\(\*\^0\^\*\)":"Excited",
    u"\(\*_\*\)":"Amazed",
    u"\(\*_\*;":"Amazed",
    u"\(\+_\+\) \(@_@\)":"Amazed",
    u"\(\*\^\^\)v":"Laughing,Cheerful",
    u"\(\^_\^\)v":"Laughing,Cheerful",
    u"\(\(d[-_-]b\)\)":"Headphones,Listening to music",
    u'\(-"-\)':"Worried",
    u"\(ーー;\)":"Worried",
    u"\(\^0_0\^\)":"Eyeglasses",
    u"\(\＾ｖ\＾\)":"Happy",
    u"\(\＾ｕ\＾\)":"Happy",
    u"\(\^\)o\(\^\)":"Happy",
    u"\(\^O\^\)":"Happy",
    u"\(\^o\^\)":"Happy",
    u"\)\^o\^\(":"Happy",
    u":O o_O":"Surprised",
    u"o_0":"Surprised",
    u"o\.O":"Surpised",
    u"\(o\.o\)":"Surprised",
    u"oO":"Surprised",
    u"\(\*￣m￣\)":"Dissatisfied",
    u"\(‘A`\)":"Snubbed or Deflated"
}

In [ ]:
def remove_emoticons(text):
    emoticon_pattern = re.compile(u'(' + u'|'.join(k for k in EMOTICONS) + u')')
    return emoticon_pattern.sub(r'', text)

In [ ]:
df_train['clean'] = df_train['clean'].apply(remove_emoticons)
df_test['clean'] = df_test['clean'].apply(remove_emoticons)

In [ ]:
!pip install indoNLP

###replace slang_function

In [ ]:
from indoNLP.preprocessing import replace_slang, replace_word_elongation, remove_stopwords

In [ ]:
def replace_slang_func(text):
    clean = replace_slang(text)
    return clean

In [ ]:
df_train['clean'] = df_train['clean'].apply(replace_slang_func)
df_test['clean'] = df_test['clean'].apply(replace_slang_func)

In [ ]:
df_train.head()

,text,sentiment,clean
0,Barang sesuai pesanan dan cepat sampai,positive,barang sesuai pesanan dan cepat sampai
1,Barang bagus harga murah,positive,barang bagus harga murah
2,Paket rapi...mantap....cepat....sampe ke tujuan,positive,paket rapi mantap cepat sampai ke tujuan
3,ya saya puas dgn barangnya,positive,ya saya puas dengan barangnya
4,Responya luar biasa b mantap,positive,responya luar biasa b mantap


In [ ]:
df_test.head()

,Unnamed: 0,text,category,product_name,product_id,clean
0,0,Warnanya tdk sesuai pesanan. Tdk ada chat/pemb...,elektronik,USB HUB 3.0 7 port by DIGIGEAR HIGH SPEED 1.2 ...,170197447,warnanya tidak sesuai pesanan tidak ada chat p...
1,1,Thanks gan barang bagus.smoga awet,handphone,Holder anti hujan &amp; copet untuk smartphone...,37564148,thanks gan barang bagus semoga awet
2,2,barang sesuai deskripsi.. laptop dapat di-charge,elektronik,Adaptor Charger Laptop Toshiba Satellite C800 ...,254339251,barang sesuai deskripsi laptop dapat di charge
3,3,Goood product.......... ............,handphone,Silikon Case Blackberry Bold 9000 Hitam Gratis...,4837401,goood product
4,4,Bagus pengiriman cepat.,handphone,Audio Splitter Jack 3.5mm to dual female U Sha...,270447288,bagus pengiriman cepat


###replace_word_elongation

In [ ]:
def replace_word_elongation_func(text):
    clean = replace_word_elongation(text)
    return clean

In [ ]:
df_train['clean'] = df_train['clean'].apply(replace_word_elongation_func)
df_test['clean'] = df_test['clean'].apply(replace_word_elongation_func)

In [ ]:
df_test.head()

,Unnamed: 0,text,category,product_name,product_id,clean
0,0,Warnanya tdk sesuai pesanan. Tdk ada chat/pemb...,elektronik,USB HUB 3.0 7 port by DIGIGEAR HIGH SPEED 1.2 ...,170197447,warnanya tidak sesuai pesanan tidak ada chat p...
1,1,Thanks gan barang bagus.smoga awet,handphone,Holder anti hujan &amp; copet untuk smartphone...,37564148,thanks gan barang bagus semoga awet
2,2,barang sesuai deskripsi.. laptop dapat di-charge,elektronik,Adaptor Charger Laptop Toshiba Satellite C800 ...,254339251,barang sesuai deskripsi laptop dapat di charge
3,3,Goood product.......... ............,handphone,Silikon Case Blackberry Bold 9000 Hitam Gratis...,4837401,goood product
4,4,Bagus pengiriman cepat.,handphone,Audio Splitter Jack 3.5mm to dual female U Sha...,270447288,bagus pengiriman cepat


###remove stop words

In [ ]:
add_sw = [
    "ya",
    "nya",
    "banget",
    "username",
    "sih",
    "kalo",
    "kayak",
    "gue",
    "deh",
    "lo",
    "lu",
    "tau",
    "biar",
    "nih",
    "sok",
    "n",
    "nah",
    "mah",
    "tuh",
    "en",
    "at",
    "eh",
    "an",
    "gan",
    "b"
]

In [ ]:
words_to_remove = ['tidak', 'apa', 'ternyata', 'ga']
stop_words_modified = [word for word in stopwords.words('indonesian') if word not in words_to_remove]

In [ ]:
stop_words_modified

['ada',
 'adalah',
 'adanya',
 'adapun',
 'agak',
 'agaknya',
 'agar',
 'akan',
 'akankah',
 'akhir',
 'akhiri',
 'akhirnya',
 'aku',
 'akulah',
 'amat',
 'amatlah',
 'anda',
 'andalah',
 'antar',
 'antara',
 'antaranya',
 'apaan',
 'apabila',
 'apakah',
 'apalagi',
 'apatah',
 'artinya',
 'asal',
 'asalkan',
 'atas',
 'atau',
 'ataukah',
 'ataupun',
 'awal',
 'awalnya',
 'bagai',
 'bagaikan',
 'bagaimana',
 'bagaimanakah',
 'bagaimanapun',
 'bagi',
 'bagian',
 'bahkan',
 'bahwa',
 'bahwasanya',
 'baik',
 'bakal',
 'bakalan',
 'balik',
 'banyak',
 'bapak',
 'baru',
 'bawah',
 'beberapa',
 'begini',
 'beginian',
 'beginikah',
 'beginilah',
 'begitu',
 'begitukah',
 'begitulah',
 'begitupun',
 'bekerja',
 'belakang',
 'belakangan',
 'belum',
 'belumlah',
 'benar',
 'benarkah',
 'benarlah',
 'berada',
 'berakhir',
 'berakhirlah',
 'berakhirnya',
 'berapa',
 'berapakah',
 'berapalah',
 'berapapun',
 'berarti',
 'berawal',
 'berbagai',
 'berdatangan',
 'beri',
 'berikan',
 'berikut',
 'beri

In [ ]:
def remove_stopword(text):
    stop_words = stop_words_modified
    stop_words.extend(add_sw)
    stop_words = set(stop_words)
    text = ' '.join([word for word in text.split() if word not in stop_words])

    return text

In [ ]:
df_train['clean'] = df_train['clean'].apply(remove_stopword)
df_test['clean'] = df_test['clean'].apply(remove_stopword)

In [ ]:
df_test

,Unnamed: 0,text,category,product_name,product_id,clean
0,0,Warnanya tdk sesuai pesanan. Tdk ada chat/pemb...,elektronik,USB HUB 3.0 7 port by DIGIGEAR HIGH SPEED 1.2 ...,170197447,warnanya tidak sesuai pesanan tidak chat pembe...
1,1,Thanks gan barang bagus.smoga awet,handphone,Holder anti hujan &amp; copet untuk smartphone...,37564148,thanks barang bagus semoga awet
2,2,barang sesuai deskripsi.. laptop dapat di-charge,elektronik,Adaptor Charger Laptop Toshiba Satellite C800 ...,254339251,barang sesuai deskripsi laptop charge
3,3,Goood product.......... ............,handphone,Silikon Case Blackberry Bold 9000 Hitam Gratis...,4837401,goood product
4,4,Bagus pengiriman cepat.,handphone,Audio Splitter Jack 3.5mm to dual female U Sha...,270447288,bagus pengiriman cepat
5,5,Terima kasih pesanannya sdh sampai,fashion,Celana jeans anak pendek,294837423,terima kasih pesanannya
6,6,"barangnya cacat, apa tidak diperiksa dulu?",handphone,Tempelan / Perekat Gurita Hp Buat / untuk Goje...,56486193,barangnya cacat apa tidak diperiksa
7,7,Barang sesuai pesanan,elektronik,Baterai Acer Aspire One 722 D255 D257 D260 D27...,189413883,barang sesuai pesanan
8,8,"Udah order ternyata stok ga tersedia, harusny...",fashion,Sepatu Bata Dairy Black - 6516353,194223236,order ternyata stok tersedia pasang etalase
9,9,Pengiriman barang lambat,elektronik,USB HUB 4 port USB 3.0 / USB HUB 3.0 &#40;4por...,182937734,pengiriman barang lambat


In [ ]:
df_train

,text,sentiment,clean
0,Barang sesuai pesanan dan cepat sampai,positive,barang sesuai pesanan cepat
1,Barang bagus harga murah,positive,barang bagus harga murah
2,Paket rapi...mantap....cepat....sampe ke tujuan,positive,paket rapi mantap cepat tujuan
3,ya saya puas dgn barangnya,positive,puas barangnya
4,Responya luar biasa b mantap,positive,responya mantap
...,...,...,...
40592,gak bisa ditutup.. ukuran gak pas..,negative,ditutup ukuran pas
40593,Penjual krg peduli dan brg jelek. Brg yg dikir...,negative,penjual krg peduli bareng jelek bareng dikirim...
40594,"pelayanan kurang sigap, berhari2 dr nanya baik...",negative,pelayanan sigap berhari kesel dikasi resi itup...
40595,"barang yg dikirim tsk sesuai wrn, kl memang dr...",negative,barang dikirim tsk sesuai warna warna tidak re...


## Tokenize

In [ ]:
def tokenise(text):
    words = word_tokenize(text)
    return words

In [ ]:
df_train['tokenized'] = df_train['clean'].apply(tokenise)
df_test['tokenized'] = df_test['clean'].apply(tokenise)

In [ ]:
df_train.head()

,text,sentiment,clean,tokenized
0,Barang sesuai pesanan dan cepat sampai,positive,barang sesuai pesanan cepat,"[barang, sesuai, pesanan, cepat]"
1,Barang bagus harga murah,positive,barang bagus harga murah,"[barang, bagus, harga, murah]"
2,Paket rapi...mantap....cepat....sampe ke tujuan,positive,paket rapi mantap cepat tujuan,"[paket, rapi, mantap, cepat, tujuan]"
3,ya saya puas dgn barangnya,positive,puas barangnya,"[puas, barangnya]"
4,Responya luar biasa b mantap,positive,responya mantap,"[responya, mantap]"


In [ ]:
df_test.head()

,Unnamed: 0,text,category,product_name,product_id,clean,tokenized
0,0,Warnanya tdk sesuai pesanan. Tdk ada chat/pemb...,elektronik,USB HUB 3.0 7 port by DIGIGEAR HIGH SPEED 1.2 ...,170197447,warnanya tidak sesuai pesanan tidak chat pembe...,"[warnanya, tidak, sesuai, pesanan, tidak, chat..."
1,1,Thanks gan barang bagus.smoga awet,handphone,Holder anti hujan &amp; copet untuk smartphone...,37564148,thanks barang bagus semoga awet,"[thanks, barang, bagus, semoga, awet]"
2,2,barang sesuai deskripsi.. laptop dapat di-charge,elektronik,Adaptor Charger Laptop Toshiba Satellite C800 ...,254339251,barang sesuai deskripsi laptop charge,"[barang, sesuai, deskripsi, laptop, charge]"
3,3,Goood product.......... ............,handphone,Silikon Case Blackberry Bold 9000 Hitam Gratis...,4837401,goood product,"[goood, product]"
4,4,Bagus pengiriman cepat.,handphone,Audio Splitter Jack 3.5mm to dual female U Sha...,270447288,bagus pengiriman cepat,"[bagus, pengiriman, cepat]"


In [ ]:
df_train.to_csv("df_train_clean.csv", index=False)
df_test.to_csv("df_test_clean.csv", index=False)

In [ ]:
df_train_clean = pd.read_csv('/content/df_train_clean.csv')
df_test_clean = pd.read_csv('/content/df_test_clean.csv')

In [ ]:
df_train_clean

,text,sentiment,clean,tokenized
0,Barang sesuai pesanan dan cepat sampai,positive,barang sesuai pesanan cepat,"['barang', 'sesuai', 'pesanan', 'cepat']"
1,Barang bagus harga murah,positive,barang bagus harga murah,"['barang', 'bagus', 'harga', 'murah']"
2,Paket rapi...mantap....cepat....sampe ke tujuan,positive,paket rapi mantap cepat tujuan,"['paket', 'rapi', 'mantap', 'cepat', 'tujuan']"
3,ya saya puas dgn barangnya,positive,puas barangnya,"['puas', 'barangnya']"
4,Responya luar biasa b mantap,positive,responya mantap,"['responya', 'mantap']"
...,...,...,...,...
40592,gak bisa ditutup.. ukuran gak pas..,negative,ditutup ukuran pas,"['ditutup', 'ukuran', 'pas']"
40593,Penjual krg peduli dan brg jelek. Brg yg dikir...,negative,penjual krg peduli bareng jelek bareng dikirim...,"['penjual', 'krg', 'peduli', 'bareng', 'jelek'..."
40594,"pelayanan kurang sigap, berhari2 dr nanya baik...",negative,pelayanan sigap berhari kesel dikasi resi itup...,"['pelayanan', 'sigap', 'berhari', 'kesel', 'di..."
40595,"barang yg dikirim tsk sesuai wrn, kl memang dr...",negative,barang dikirim tsk sesuai warna warna tidak re...,"['barang', 'dikirim', 'tsk', 'sesuai', 'warna'..."


In [ ]:
df_test_clean

,Unnamed: 0,text,category,product_name,product_id,clean,tokenized
0,0,Warnanya tdk sesuai pesanan. Tdk ada chat/pemb...,elektronik,USB HUB 3.0 7 port by DIGIGEAR HIGH SPEED 1.2 ...,170197447,warnanya tidak sesuai pesanan tidak chat pembe...,"['warnanya', 'tidak', 'sesuai', 'pesanan', 'ti..."
1,1,Thanks gan barang bagus.smoga awet,handphone,Holder anti hujan &amp; copet untuk smartphone...,37564148,thanks barang bagus semoga awet,"['thanks', 'barang', 'bagus', 'semoga', 'awet']"
2,2,barang sesuai deskripsi.. laptop dapat di-charge,elektronik,Adaptor Charger Laptop Toshiba Satellite C800 ...,254339251,barang sesuai deskripsi laptop charge,"['barang', 'sesuai', 'deskripsi', 'laptop', 'c..."
3,3,Goood product.......... ............,handphone,Silikon Case Blackberry Bold 9000 Hitam Gratis...,4837401,goood product,"['goood', 'product']"
4,4,Bagus pengiriman cepat.,handphone,Audio Splitter Jack 3.5mm to dual female U Sha...,270447288,bagus pengiriman cepat,"['bagus', 'pengiriman', 'cepat']"
5,5,Terima kasih pesanannya sdh sampai,fashion,Celana jeans anak pendek,294837423,terima kasih pesanannya,"['terima', 'kasih', 'pesanannya']"
6,6,"barangnya cacat, apa tidak diperiksa dulu?",handphone,Tempelan / Perekat Gurita Hp Buat / untuk Goje...,56486193,barangnya cacat apa tidak diperiksa,"['barangnya', 'cacat', 'apa', 'tidak', 'diperi..."
7,7,Barang sesuai pesanan,elektronik,Baterai Acer Aspire One 722 D255 D257 D260 D27...,189413883,barang sesuai pesanan,"['barang', 'sesuai', 'pesanan']"
8,8,"Udah order ternyata stok ga tersedia, harusny...",fashion,Sepatu Bata Dairy Black - 6516353,194223236,order ternyata stok tersedia pasang etalase,"['order', 'ternyata', 'stok', 'tersedia', 'pas..."
9,9,Pengiriman barang lambat,elektronik,USB HUB 4 port USB 3.0 / USB HUB 3.0 &#40;4por...,182937734,pengiriman barang lambat,"['pengiriman', 'barang', 'lambat']"


In [ ]:
df_train_clean.isnull().sum()

text           0
sentiment      0
clean        244
tokenized      0
dtype: int64

In [ ]:
df_train_clean = df_train_clean.dropna()
df_train_clean.isnull().sum()

text         0
sentiment    0
clean        0
tokenized    0
dtype: int64

# BoW

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df_train_clean['tokenized'])
X = X.toarray()

In [ ]:
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
sorted(vectorizer.vocabulary_.keys())

['aaaaaaaaaaaaaaaat',
 'aaaaaaaaaap',
 'aaajajajajajajakkakakakaka',
 'aaamiin',
 'aah',
 'aalah',
 'aamiiiin',
 'aan',
 'aaya',
 'abal',
 'abalterimakasih',
 'abang',
 'abangnya',
 'abcdefghijklmnopqrstuvwxyz',
 'abdi',
 'abieas',
 'abiis',
 'abis',
 'abisz',
 'abiz',
 'about',
 'absen',
 'absolutely',
 'abu',
 'abux',
 'ac',
 'acara',
 'accept',
 'acceptable',
 'acces',
 'accesories',
 'according',
 'ace',
 'aceh',
 'acepet',
 'acer',
 'ach',
 'acold',
 'act',
 'action',
 'active',
 'activities',
 'acuan',
 'adahargaadakualitas',
 'adapter',
 'adaptor',
 'adaptornya',
 'adaptory',
 'adeeeem',
 'adeeem',
 'adek',
 'adem',
 'adfgjnb',
 'adha',
 'adidas',
 'adik',
 'adil',
 'adipiscing',
 'adjana',
 'adjust',
 'adjustment',
 'adjutable',
 'admiin',
 'admin',
 'adminnya',
 'aduh',
 'advan',
 'advance',
 'advertised',
 'adwem',
 'adzkareload',
 'aerobic',
 'aesthetic',
 'aesuai',
 'affordable',
 'afiat',
 'after',
 'afteral',
 'aftersale',
 'aftersales',
 'ag',
 'aga',
 'agaaan',
 'agabsg

In [ ]:
import pickle

with open('count_vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)
    pickle.dump(X, f)

# TD-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
transformed = tfidf.fit_transform(df_train_clean['tokenized'])

In [ ]:
df_tfidf= pd.DataFrame(transformed[0].T.todense(), index=tfidf.get_feature_names_out(), columns=["TF-IDF"])
df_tfidf = df_tfidf.sort_values('TF-IDF', ascending=False)
df_tfidf

,TF-IDF
pesanan,0.675600
cepat,0.464259
sesuai,0.431347
barang,0.376786
penanganannya,0.000000
...,...
hdhdhshe,0.000000
hdmi,0.000000
hdminya,0.000000
he,0.000000


In [ ]:
import pickle
with open('tfidf_dataframe.pkl', 'wb') as f:
    pickle.dump(df_tfidf, f)

# Pemodelan

## Split Train Test

In [ ]:
#Vectorization for Modelling

vectorizer_mod = TfidfVectorizer(max_features=5000)

In [ ]:
X = vectorizer_mod.fit_transform(df_train_clean['tokenized']).toarray()
y = df_train_clean['sentiment']

In [ ]:
from collections import Counter

counter_y = Counter(y)
print(counter_y)

Counter({'positive': 37639, 'neutral': 1798, 'negative': 916})


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

## Modelling

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold, GridSearchCV


classifiers = {
    'LogisiticRegression': LogisticRegression(max_iter=2500, solver='saga'),
    'KNearest': KNeighborsClassifier(),
}

for key, classifier in classifiers.items():
    classifier.fit(X_train, y_train)
    training_score = cross_val_score(classifier, X_train, y_train, cv=5)
    print(classifier.__class__.__name__, "=", training_score.mean() * 100, "% accuracy score")

LogisticRegression = 93.48552736346193 % accuracy score
KNeighborsClassifier = 92.95582788743147 % accuracy score


In [ ]:
model = LogisticRegression(max_iter=2500, solver='saga')
model.fit(X_train,y_train)

LogisticRegression(max_iter=2500, solver='saga')

In [ ]:
with open('LR.pkl', 'wb') as file:
    pickle.dump(model, file)

In [ ]:
y_preds = model.predict(X_test)

print(confusion_matrix(y_test,y_preds))
print(classification_report(y_test,y_preds))
print('nilai akurasinya adalah ',accuracy_score(y_test, y_preds))

[[  52    4  123]
 [   9    0  328]
 [  15   10 7530]]
              precision    recall  f1-score   support

    negative       0.68      0.29      0.41       179
     neutral       0.00      0.00      0.00       337
    positive       0.94      1.00      0.97      7555

    accuracy                           0.94      8071
   macro avg       0.54      0.43      0.46      8071
weighted avg       0.90      0.94      0.92      8071

nilai akurasinya adalah  0.9394127121794078


In [ ]:
test = vectorizer_mod.transform(df_test_clean["tokenized"]).toarray()
y_classification = model.predict(test)

df_test["Sentiment"] = list(y_classification)

In [ ]:
df_test

,Unnamed: 0,text,category,product_name,product_id,clean,tokenized,Sentiment
0,0,Warnanya tdk sesuai pesanan. Tdk ada chat/pemb...,elektronik,USB HUB 3.0 7 port by DIGIGEAR HIGH SPEED 1.2 ...,170197447,warnanya tidak sesuai pesanan tidak chat pembe...,"[warnanya, tidak, sesuai, pesanan, tidak, chat...",positive
1,1,Thanks gan barang bagus.smoga awet,handphone,Holder anti hujan &amp; copet untuk smartphone...,37564148,thanks barang bagus semoga awet,"[thanks, barang, bagus, semoga, awet]",positive
2,2,barang sesuai deskripsi.. laptop dapat di-charge,elektronik,Adaptor Charger Laptop Toshiba Satellite C800 ...,254339251,barang sesuai deskripsi laptop charge,"[barang, sesuai, deskripsi, laptop, charge]",positive
3,3,Goood product.......... ............,handphone,Silikon Case Blackberry Bold 9000 Hitam Gratis...,4837401,goood product,"[goood, product]",positive
4,4,Bagus pengiriman cepat.,handphone,Audio Splitter Jack 3.5mm to dual female U Sha...,270447288,bagus pengiriman cepat,"[bagus, pengiriman, cepat]",positive
5,5,Terima kasih pesanannya sdh sampai,fashion,Celana jeans anak pendek,294837423,terima kasih pesanannya,"[terima, kasih, pesanannya]",positive
6,6,"barangnya cacat, apa tidak diperiksa dulu?",handphone,Tempelan / Perekat Gurita Hp Buat / untuk Goje...,56486193,barangnya cacat apa tidak diperiksa,"[barangnya, cacat, apa, tidak, diperiksa]",positive
7,7,Barang sesuai pesanan,elektronik,Baterai Acer Aspire One 722 D255 D257 D260 D27...,189413883,barang sesuai pesanan,"[barang, sesuai, pesanan]",positive
8,8,"Udah order ternyata stok ga tersedia, harusny...",fashion,Sepatu Bata Dairy Black - 6516353,194223236,order ternyata stok tersedia pasang etalase,"[order, ternyata, stok, tersedia, pasang, etal...",positive
9,9,Pengiriman barang lambat,elektronik,USB HUB 4 port USB 3.0 / USB HUB 3.0 &#40;4por...,182937734,pengiriman barang lambat,"[pengiriman, barang, lambat]",positive
